In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report


print("Imported successfully!")

Imported successfully!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)),
])

In [ ]:
data_dir = "fruits and vegies\\archive (1)"

train_dataset = torchvision.datasets.ImageFolder(root=f"{data_dir}\\train", transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=f"{data_dir}\\test", transform=transform_test)
val_dataset = torchvision.datasets.ImageFolder(root=f"{data_dir}\\validation", transform=transform_test)

In [10]:
print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Test samples:", len(test_dataset))
print("Classes:", train_dataset.classes)


Train samples: 46187
Validation samples: 12544
Test samples: 13353
Classes: ['Apple', 'Avocado', 'Banana', 'Beetroot', 'Blackberry', 'Blueberry', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Chilli Peper', 'Corn', 'Cucumber', 'Dates', 'Dragonfruit', 'Eggplant', 'Fig', 'Garlic', 'Ginger', 'Grapes', 'Guava', 'Jalepeno', 'Kiwi', 'Lemon', 'Lettuce', 'Mango', 'Mushroom', 'Okra', 'Olive', 'Onion', 'Orange', 'Paprika', 'Peanuts', 'Pear', 'Peas', 'Pineapple', 'Pomegranate', 'Potato', 'Pumpkin', 'Raddish', 'Rambutan', 'Soy Beans', 'Spinach', 'Strawberry', 'Sweetcorn', 'Sweetpotato', 'Tomato', 'Turnip', 'Watermelon']


In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers = 4)


In [22]:
image, label = train_dataset[0]
image.size()

torch.Size([3, 224, 224])

In [ ]:
class_names = train_dataset.classes
class_names



['Apple',
 'Avocado',
 'Banana',
 'Beetroot',
 'Blackberry',
 'Blueberry',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Chilli Peper',
 'Corn',
 'Cucumber',
 'Dates',
 'Dragonfruit',
 'Eggplant',
 'Fig',
 'Garlic',
 'Ginger',
 'Grapes',
 'Guava',
 'Jalepeno',
 'Kiwi',
 'Lemon',
 'Lettuce',
 'Mango',
 'Mushroom',
 'Okra',
 'Olive',
 'Onion',
 'Orange',
 'Paprika',
 'Peanuts',
 'Pear',
 'Peas',
 'Pineapple',
 'Pomegranate',
 'Potato',
 'Pumpkin',
 'Raddish',
 'Rambutan',
 'Soy Beans',
 'Spinach',
 'Strawberry',
 'Sweetcorn',
 'Sweetpotato',
 'Tomato',
 'Turnip',
 'Watermelon']

In [25]:
len(class_names)

50

In [ ]:
class netCNN(nn.Module):
    def __init__(self, num_classes):
        super(netCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling → output size 4x4 (can be adjusted)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # Convolution + BatchNorm + ReLU + Pool
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 32 x 112 x 112
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # 64 x 56 x 56
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # 128 x 28 x 28
        
        # Adaptive Pooling → 128 x 4 x 4
        x = self.adaptive_pool(x)
        
        # Flatten
        x = x.view(-1, 128 * 4 * 4)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [ ]:
net = netCNN(num_classes=len(class_names)).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 20
best_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)

    # ------------------ TRAIN ------------------
    net.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_acc = 100 * correct_train / total_train
    train_loss /= len(train_loader)

    # ------------------ VALIDATION ------------------
    net.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_acc = 100 * correct_val / total_val
    val_loss /= len(val_loader)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

    # save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(net.state_dict(), "best_model.pth")
        print(f"🔥 Saved new BEST model (Acc={best_acc:.2f}%)")

    # update LR
    scheduler.step()

print("\nTraining finished!")
print(f"Best validation accuracy: {best_acc:.2f}%")

In [ ]:
net.load_state_dict(torch.load("best_model.pth"))
net.eval()

test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total_test += labels.size(0)
        correct_test += predicted.eq(labels).sum().item()

test_acc = 100 * correct_test / total_test
test_loss /= len(test_loader)

print("\n========== TEST RESULTS ==========")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")
print("=================================\n")



In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = net(images)
        _, preds = outputs.max(1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:\n", cm)

In [ ]:
# =============================================================
# ---------------------- EXPORT TO ONNX -----------------------
# =============================================================

dummy_input = torch.randn(1, 3, 128, 128).to(device)
torch.onnx.export(
    net,
    dummy_input,
    "model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)

print("\nModel exported to model.onnx")